
# Calculation of auditory features
This notebook calculates the auditory features for any sound file that are used in

the publication Arato & Fitch (2021): 

Phylogenetic Signal in the Vocalizations of Vocal Learning and Vocal Non-Learning Birds


Dependencies: 

numpy

scipy 

pandas

matplotlib

librosa 

entropy package: https://github.com/raphaelvallat/entropy


# Import Libraries

In [10]:
import numpy as np
from scipy.signal import butter, lfilter
import pandas as pd
import matplotlib.pyplot as plt
import librosa
from librosa import display
from EntropyMod import spectral_entropy



#  Preprocessing  + Acoustic feature calculation functions 

In [15]:


def BandFilt(Input,sr,lowHz,highHz):
    Nyquist = sr * 0.5
    # first define the cutoff freqs, in relation to Nyquist:
    low = lowHz/Nyquist
    high = highHz/Nyquist
    b,a = butter(5, [low, high], btype='band')
    return lfilter(b, a, Input)

def Calc_SpectralFlux(X):

    # difference spectrum (set first diff to zero)
    X = np.c_[X[:, 0], X]
    afDeltaX = np.diff(X, 1, axis=1)

    # flux
    vsf = np.sqrt((afDeltaX**2).sum(axis=0)) / X.shape[0]

    return np.log(np.std(vsf))


def Calc_Flatness(yy):
    SpecFlat=librosa.feature.spectral_flatness(y=yy)
    return np.log(np.mean(SpecFlat))

def CV_RMS(Spect,n_fft,hop_length):
    RMStr=librosa.feature.rms(S=Spect,frame_length=n_fft,hop_length=hop_length)
    return np.std(RMStr)/np.mean(RMStr)

def Calc_ContrastBands(Spect,sr,nband):
    Contr=librosa.feature.spectral_contrast(S=Spect,sr=sr,n_bands=nband-1,fmin=500)  # fmin is the first band cutoff [0:fmin]
    return np.log(np.mean(Contr,1))


def Calc_SpectCentroid(Spect,sr):
    SpectCentr=librosa.feature.spectral_centroid(S=Spect,sr=sr)[0]
    return np.log(np.median(SpectCentr))



#  Function for performing preprocessing and calculating all measures for a sound file

In [16]:
def CalcMeasures(y,Fs,n_fft,hop_length,Zscore=1, NContrastBands=4,VisSpecto=0):
    ''' this function performs preprocessing of auditory signal and returns values of acoustic features for the whole signal'''
    y=BandFilt(y,Fs,100,10000) 
    if Zscore==1:
        y=(y-np.mean(y))/np.std(y)
    Specto = np.abs(librosa.stft(y, n_fft=n_fft,hop_length=hop_length)) 
    if VisSpecto==True:
        display.specshow(Specto,cmap='gray_r')
    SpecFl=Calc_SpectralFlux(Specto)
    Flat=Calc_Flatness(y)
    RMS=CV_RMS(Specto,n_fft,hop_length)
    Contrast=Calc_ContrastBands(Specto,Fs,NContrastBands)
    Centroid=Calc_SpectCentroid(Specto,Fs)
    Entro=spectral_entropy(y, Fs, method='fft', nperseg=None, normalize=False)
     
    return RMS,SpecFl,Entro,Flat,Centroid,Contrast,len(y)/Fs



#  Visualize spectogram and calculate all measures for an audio file

In [ ]:

n_fft = 512
hop_length = 256
NBands=4  # number of contrast bands

FileName=# YOUR AUDIO FILE
y, sr = librosa.load(FileName)

RMS,SpecFl,Entro,Flat,Centroid,Contrast,Length=CalcMeasures(y,sr,n_fft,hop_length,NContrastBands=NBands,VisSpecto=1)
